#### CMA100， 空间宽带声源定位测试
1. 该算法预用于俯仰角抑制后，希望能够在我们关注的区域内找到声源位置。
2. 测试包括单声源测试， 以及多声源测试。
3. 在多声源测试中，引入了一个语音源和噪声源，他们的长度都为4s， 以512个采样点为一帧(32ms),帧移128个采样点(延时8ms)。可以控制他们的总体信噪比， 但每一帧的信噪比不一样，有些帧中噪声能量会超过信号能量，这一部分还需要考虑一些额外处理。
4. 对于单声源测试， 测试效果非常稳定且准确。

In [1]:
from Beamform.BFModel import beamforming_model
from Beamform.CMA100 import CMA100
import librosa
from Beamform.tools import ConvertToTargetDb, CalRms, NormLength
import numpy as np
import IPython.display as ipd
import soundfile as sf

In [2]:
# 初始化cma100阵列
cma100 = CMA100(fs=16000, L=512)
# 导入音频
audio_data, _ = librosa.load('./test_audios/1.wav', sr=16000)
noise_data, _ = librosa.load('./test_audios/A8_0.wav', sr=16000)
audio_data = audio_data[:4*16000]
noise_data = noise_data[:4*16000]
# 控制声音能量
audio_data = ConvertToTargetDb(audio_data, -20)
noise_data = ConvertToTargetDb(noise_data, -30)
sf.write("./audio_data.wav", audio_data, samplerate=16000)
sf.write("./noise_data.wav", noise_data, samplerate=16000)

audio_rfft = librosa.stft(audio_data, n_fft=512, win_length=512, hop_length=128)
noise_rfft = librosa.stft(noise_data, n_fft=512, win_length=512, hop_length=128)


In [3]:
# 空间单信号源定位测试， 对单一信号源进行定位测试：
# 设置声源位置
audio_x = 3.0
audio_y = 0.0
audio_z = 0.0

# 模拟阵列流式接受信号， 并处理的过程
frameLength = 512
N = audio_rfft.shape[1]
enhancedAudio = np.zeros_like(audio_rfft)
micAudio = np.zeros_like(audio_rfft)

# 流式处理：
for i in range(N):
    cma100.add_signal(S_x = audio_x, S_y = audio_y, S_z = audio_z, signal_rfft = audio_rfft[:, i])
    cma100.band_filter() # 频点分配
    cma100.angleFilter_doa() # 角度抑制 + doa
    cma100.reset() # 重置阵列，用于处理下一帧
    if i == 15:
        break

KeyboardInterrupt: 

In [4]:
# 空间单信号源定位测试， 对单一信号源进行定位测试：
# 设置声源位置
audio_x = 3.0
audio_y = 0.0
audio_z = 0.0

# 模拟阵列流式接受信号， 并处理的过程
frameLength = 512
N = audio_rfft.shape[1]
enhancedAudio = np.zeros_like(audio_rfft)
micAudio = np.zeros_like(audio_rfft)

# 流式处理：
for i in range(N):
    cma100.reset()
    cma100.add_signal(S_x = audio_x, S_y = audio_y, S_z = audio_z, signal_rfft = audio_rfft[:, i])
    cma100.band_filter() # 频点分配
    cma100.angleFilter_doa(bf_method = "music") # 角度抑制 + doa
    if i == 15:
        break

x is 0.0
y is -0.0
x is 3.0
y is 0.0
x is 3.0
y is 0.0


KeyboardInterrupt: 

In [5]:
# 空间双信号源定位测试， 对能量较大的信号源进行定位测试：
# 设置声源位置
audio_x = 3.0
audio_y = 0.0
audio_z = 0.0

noise_x = -2.0
noise_y = 1.0
noise_z = 0.0

# 模拟阵列流式接受信号， 并处理的过程
frameLength = 512
N = audio_rfft.shape[1]
enhancedAudio = np.zeros_like(audio_rfft)
micAudio = np.zeros_like(audio_rfft)


# 流式处理：
for i in range(N):
    cma100.add_signal(S_x = audio_x, S_y = audio_y, S_z = audio_z, signal_rfft = audio_rfft[:, i])
    cma100.add_signal(S_x = noise_x, S_y = noise_y, S_z = noise_z, signal_rfft = noise_rfft[:, i])
    micAudio[:,i] = cma100.build_signal_rfft(method=0) # 把处理前的信号提取出来
    cma100.band_filter() # 频点分配
    cma100.angleFilter_doa(bf_method="music") # 角度抑制
    #cma100.angleFilter_Beamform()
    #enhancedAudio[:,i] = cma100.build_signal_rfft(method=0) # 将处理后的信号提取出来
    cma100.reset() # 重置阵列，用于处理下一帧
    if i == 10: break

x is 3.0
y is 0.0
x is 3.0
y is 0.0
x is 3.0
y is 0.0
x is 3.0
y is 0.0
x is 3.0
y is 0.0
x is 3.0
y is 0.0
x is 3.0
y is 0.0
x is 3.0
y is 0.0
x is 3.0
y is 0.0
x is 3.0
y is 0.0
x is 3.0
y is 0.0


In [6]:
# 空间双信号源定位测试， 对能量较大的信号源进行定位测试：
# 设置声源位置
audio_x = 3.0
audio_y = 0.0
audio_z = 0.0

noise_x = -2.0
noise_y = 1.0
noise_z = 0.0

# 模拟阵列流式接受信号， 并处理的过程
frameLength = 512
N = audio_rfft.shape[1]
enhancedAudio = np.zeros_like(audio_rfft)
micAudio = np.zeros_like(audio_rfft)


# 流式处理：
for i in range(N):
    cma100.add_signal(S_x = audio_x, S_y = audio_y, S_z = audio_z, signal_rfft = audio_rfft[:, i])
    cma100.add_signal(S_x = noise_x, S_y = noise_y, S_z = noise_z, signal_rfft = noise_rfft[:, i])
    micAudio[:,i] = cma100.build_signal_rfft(method=0) # 把处理前的信号提取出来
    cma100.band_filter() # 频点分配
    cma100.angleFilter_doa(bf_method="cbf") # 角度抑制
    #cma100.angleFilter_Beamform()
    #enhancedAudio[:,i] = cma100.build_signal_rfft(method=0) # 将处理后的信号提取出来
    cma100.reset() # 重置阵列，用于处理下一帧
    if i == 10: break

x is 0.0
y is -0.0
x is 3.0
y is 0.0
x is 3.0
y is 0.0
x is 3.0
y is 0.0
x is 3.0
y is 0.0
x is 3.0
y is 0.0
x is 3.0
y is 0.0
x is 3.0
y is 0.0
x is 2.899558315953535
y is 0.05061197866812226
x is 2.9000000000000004
y is 0.0
x is 3.0
y is 0.0
